# Skip-gram and negative sampling

In [2]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

### function that can be called on a list of vectorized sentences obtained from any text dataset. 

In [3]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

### Prepare training data for word2vec

In [9]:
def remove_empty_lines(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Filter out empty lines
    non_empty_lines = [line.strip() for line in lines if line.strip()]

    with open(filename, 'w') as file:
        file.write('\n'.join(non_empty_lines))

filename = 'abstracts.txt'
remove_empty_lines(filename)
print("Empty lines removed from", filename)


Empty lines removed from abstracts.txt


In [10]:
with open(filename) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

ERK1 and ERK2 are related protein-serine/threonine kinases that participate in the Ras-Raf-MEK-ERK signal transduction cascade. This cascade participates in the regulation of a large variety of processes including cell adhesion, cell cycle progression, cell migration, cell survival, differentiation, metabolism, proliferation, and transcription. MEK1/2 catalyze the phosphorylation of human ERK1/2 at Tyr204/187 and then Thr202/185. The phosphorylation of both tyrosine and threonine is required for enzyme activation. Whereas the Raf kinase and MEK families have narrow substrate specificity, ERK1/2 catalyze the phosphorylation of hundreds of cytoplasmic and nuclear substrates including regulatory molecules and transcription factors. ERK1/2 are proline-directed kinases that preferentially catalyze the phosphorylation of substrates containing a Pro-Xxx-Ser/Thr-Pro sequence. Besides this primary structure requirement, many ERK1/2 substrates possess a D-docking site, an F-docking site, or both

### loads text from text files and creates a dataset where each line of the files becomes an element of the dataset.

In [17]:
text_ds = tf.data.TextLineDataset(filename).filter(lambda x: tf.cast(tf.strings.length(x), bool))

### vectorize sentences

In [20]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [21]:
vectorize_layer.adapt(text_ds.batch(1024))

2024-02-16 23:31:48.103581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [22]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'of', 'and', 'in', 'to', 'a', 'for', 'with', 'that', 'is', 'by', 'we', 'were', 'as', 'was', 'are', 'from', 'on']


In [24]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

### obtain sequences from the dataset

I now have a tf.data.Dataset of integer encoded sentences. To prepare the dataset for training a word2vec model, flatten the dataset into a list of sentence vector sequences. This step is required as I would iterate over each sentence in the dataset to produce positive and negative examples.

In [25]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

20010


In [26]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[3697    4 2793   17  228    1 1917   10 2726    5] => ['erk1', 'and', 'erk2', 'are', 'related', '[UNK]', 'kinases', 'that', 'participate', 'in']
[   5   20   30   13  128  105 1311   63    2  175] => ['in', 'this', 'study', 'we', 'provide', 'novel', 'insight', 'into', 'the', 'mechanism']
[   2    1   28  512  666    1 1917    1   17 1142] => ['the', '[UNK]', 'protein', 'map', 'kinase', '[UNK]', 'kinases', '[UNK]', 'are', 'activated']
[   2 1255 1023  666  111    4  125    1    7  685] => ['the', 'extracellular', 'regulated', 'kinase', '1', 'and', '2', '[UNK]', 'a', 'variety']
[   2    1 1917 3697    4 2793 1198 2376    6   15] => ['the', '[UNK]', 'kinases', 'erk1', 'and', 'erk2', 'commonly', 'referred', 'to', 'as']


### Generating training examples from sequences

sequences is now a list of int encoded sentences. Just call the generate_training_data function defined earlier to generate training examples for the word2vec model.

In [28]:
SEED = 42

targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████████████████████████████████████████| 20010/20010 [00:55<00:00, 362.97it/s]




targets.shape: (60678,)
contexts.shape: (60678, 5)
labels.shape: (60678, 5)


### configure the dataset for performance

To perform efficient batching for the potentially large number of training examples, use the tf.data.Dataset API. After this step, I would have a tf.data.Dataset object of (target_word, context_word), (label) elements to train my word2vec model!

In [29]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


Apply Dataset.cache and Dataset.prefetch to improve performance:

In [30]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


# Model and training

The word2vec model can be implemented as a classifier to distinguish between true context words from skip-grams and false context words obtained through negative sampling. 

In [34]:
# Set the number of negative samples per positive context.
num_ns = 4

class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

### define loss function and compile model

In [32]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

instantiate my word2vec class with an embedding dimension of 128. compile the model with the tf.keras.optimizers.Adam optimizer.

In [35]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

Also define a callback to log training statistics for TensorBoard:

In [36]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Train the model on the dataset for some number of epochs:

In [37]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
59/59 [==============================] - 4s 34ms/step - loss: 1.6057 - accuracy: 0.2969
Epoch 2/20
59/59 [==============================] - 1s 17ms/step - loss: 1.5650 - accuracy: 0.6058
Epoch 3/20
59/59 [==============================] - 1s 17ms/step - loss: 1.4597 - accuracy: 0.5710
Epoch 4/20
59/59 [==============================] - 1s 17ms/step - loss: 1.3383 - accuracy: 0.5545
Epoch 5/20
59/59 [==============================] - 1s 17ms/step - loss: 1.2375 - accuracy: 0.5836
Epoch 6/20
59/59 [==============================] - 1s 17ms/step - loss: 1.1450 - accuracy: 0.6246
Epoch 7/20
59/59 [==============================] - 1s 17ms/step - loss: 1.0587 - accuracy: 0.6651
Epoch 8/20
59/59 [==============================] - 1s 17ms/step - loss: 0.9796 - accuracy: 0.7024
Epoch 9/20
59/59 [==============================] - 1s 18ms/step - loss: 0.9075 - accuracy: 0.7317
Epoch 10/20
59/59 [==============================] - 1s 17ms/step - loss: 0.8417 - accuracy: 0.7577
Epoch 11/

TensorBoard now shows the word2vec model's accuracy and loss:

In [39]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [40]:
#docs_infra: no_execute
%tensorboard --logdir logs

### Embedding lookup and analysis

Obtain the weights from the model using Model.get_layer and Layer.get_weights. The TextVectorization.get_vocabulary function provides the vocabulary to build a metadata file with one token per line.

In [41]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Create and save the vectors and metadata files to analyze the obtained embeddings in the Embedding Projector:

https://projector.tensorflow.org

In [42]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [43]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

In [45]:
import IPython
url = 'https://projector.tensorflow.org/'

IPython.display.IFrame(url, width=1333, height=900)

Need to load files